<a href="https://colab.research.google.com/github/frankuc19/fran/blob/main/examples/notebook/constraint_solver/vrp_time_windows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# vrp_time_windows

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/main/examples/notebook/constraint_solver/vrp_time_windows.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/main/ortools/constraint_solver/samples/vrp_time_windows.py"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [1]:
%pip install ortools
!pip install h3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.4 MB/s eta 0:00:00



Vehicles Routing Problem (VRP) with Time Windows.


In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp



def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["time_matrix"] = [
        [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
        [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
        [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
        [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
        [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
        [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
        [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
        [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
        [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
        [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
        [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
        [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
        [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
        [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
        [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
        [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
        [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
    ]
    data["time_windows"] = [
        (0, 5),  # depot
        (7, 12),  # 1
        (10, 15),  # 2
        (16, 18),  # 3
        (10, 13),  # 4
        (0, 5),  # 5
        (5, 10),  # 6
        (0, 4),  # 7
        (5, 10),  # 8
        (0, 3),  # 9
        (10, 16),  # 10
        (10, 15),  # 11
        (0, 5),  # 12
        (5, 10),  # 13
        (7, 8),  # 14
        (10, 15),  # 15
        (11, 15),  # 16
    ]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")


def main():
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        30,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data["time_windows"]):
        if location_idx == data["depot"]:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data["depot"]
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data["time_windows"][depot_idx][0], data["time_windows"][depot_idx][1]
        )

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


main()



In [13]:
# ----------------------------------------------------------
# Parte 1: Imports y Funciones del Script de Asignación
# ----------------------------------------------------------
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import math
import h3 # Asegúrate de que h3 está instalado (!pip install h3) # H3 real no se usa, se simula
from tqdm import tqdm # Para la barra de progreso
import os # Para verificar existencia de archivos
import time # Para medir tiempos (opcional)

# --- Importar para descarga en Colab ---
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
# -------------------------------------

import ortools # <--- AÑADE O ASEGÚRATE QUE ESTÉ ESTA LÍNEA
# --- Imports de OR-Tools ---
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# ---------------------------


# ----------------------------------------------------------
# Parte 2: Constantes y Configuraciones Globales
# ----------------------------------------------------------
RADIO_TIERRA_KM = 6371
PRECISION_H3_SIMULADA = 3 # Precisión para la simulación de H3
DEFAULT_AVG_SPEED_KMH = 40

# Parámetros del Modelo (¡Ajusta según sea necesario!)
PARAM_MAX_MOVILES = 100
PARAM_MAX_RESERVAS_POR_MOVIL = 5
PARAM_MAX_HORAS_POR_MOVIL = 10
ID_COLUMN_NAME = 'job_id' # <--- ¡¡¡ VERIFICA ESTE NOMBRE EN TU CSV DE PREDICCIONES !!!
SOLVER_TIME_LIMIT_SECONDS = 60
MAX_SLACK_MINUTES = 30
PICKUP_WINDOW_DURATION_MINUTES = 60

# Valor grande para representar infinito (en minutos)
INF_TIME = 30 * 24 * 60 # 30 días en minutos

# Variables globales para datos precalculados
h3_time_lookup = {} # Diccionario para búsqueda rápida O(1)
avg_speed_kmh = DEFAULT_AVG_SPEED_KMH

# ----------------------------------------------------------
# Parte 3: Funciones de Cálculo (optimizadas donde es posible)
# ----------------------------------------------------------

def haversine(lat1, lon1, lat2, lon2):
    """Calcula la distancia Haversine entre dos puntos en KM."""
    # Manejo de NaNs al inicio
    if pd.isna(lat1) or pd.isna(lon1) or pd.isna(lat2) or pd.isna(lon2):
        return np.nan

    # Convertir a radianes
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Fórmula Haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = RADIO_TIERRA_KM * c
    return distance

def simulate_h3_str(lat, lon, precision=PRECISION_H3_SIMULADA):
     """Genera un ID H3 string simulado basado en lat/lon redondeados."""
     if pd.isna(lat) or pd.isna(lon):
         return None
     # Usar f-string para eficiencia
     return f"{round(lat, precision)}_{round(lon, precision)}"

def precompute_historical_averages(df_hist):
    """
    Calcula tiempos promedio desde datos históricos y velocidad fallback.
    Almacena los promedios en un diccionario global para búsqueda rápida.
    """
    global h3_time_lookup, avg_speed_kmh
    required_hist_cols = {'latrecogida', 'lonrecogida', 'latdestino', 'londestino', 'tiempoestimada'}

    print("Iniciando pre-cómputo de promedios históricos...")
    start_time = time.time()

    if not required_hist_cols.issubset(df_hist.columns):
        missing = required_hist_cols - set(df_hist.columns)
        raise ValueError(f"El DataFrame histórico no tiene las columnas necesarias: {missing}")

    print("Limpiando datos históricos...")
    initial_rows = len(df_hist)
    # Limpiar NaNs en columnas clave y convertir tiempo a numérico
    df_hist = df_hist.dropna(subset=list(required_hist_cols))
    df_hist['tiempoestimada'] = pd.to_numeric(df_hist['tiempoestimada'], errors='coerce')
    df_hist = df_hist.dropna(subset=['tiempoestimada'])
    df_hist = df_hist[df_hist['tiempoestimada'] > 0] # Asegurar tiempo positivo
    cleaned_rows_1 = len(df_hist)
    print(f"  {initial_rows - cleaned_rows_1} filas eliminadas por NaNs o tiempo inválido.")

    if df_hist.empty:
        print("Advertencia: No hay datos históricos válidos después de la limpieza inicial.")
        h3_time_lookup = {}
        avg_speed_kmh = DEFAULT_AVG_SPEED_KMH
        print(f"Usando velocidad fallback por defecto: {avg_speed_kmh} km/h")
        return

    print("Calculando H3 simulados y distancias para datos históricos...")
    # Usar .apply (más rápido que iterrows)
    df_hist['h3_origin'] = df_hist.apply(
        lambda row: simulate_h3_str(row['latrecogida'], row['lonrecogida']), axis=1
    )
    df_hist['h3_destino'] = df_hist.apply(
        lambda row: simulate_h3_str(row['latdestino'], row['londestino']), axis=1
    )
    # Vectorizar cálculo de distancia si es posible (haversine no es trivial de vectorizar directamente en pandas apply)
    # Mantenemos apply por simplicidad aquí, pero se podría optimizar más si fuera necesario
    df_hist['distance_km'] = df_hist.apply(
        lambda row: haversine(row['latrecogida'], row['lonrecogida'], row['latdestino'], row['londestino']), axis=1
    )

    # Filtrar filas donde H3 o distancia no se pudieron calcular o son inválidas
    df_hist = df_hist.dropna(subset=['h3_origin', 'h3_destino', 'distance_km'])
    df_hist = df_hist[df_hist['distance_km'] > 0.01] # Filtrar distancias muy pequeñas/inválidas
    cleaned_rows_2 = len(df_hist)
    print(f"  {cleaned_rows_1 - cleaned_rows_2} filas eliminadas por cálculo inválido de H3/distancia.")

    if df_hist.empty:
        print("Advertencia: No hay datos históricos válidos después de calcular H3/distancia.")
        h3_time_lookup = {}
        avg_speed_kmh = DEFAULT_AVG_SPEED_KMH
        print(f"Usando velocidad fallback por defecto: {avg_speed_kmh} km/h")
        return

    print("Calculando tiempos promedio (mediana) agrupados por H3...")
    # Calcular mediana de tiempos por par H3
    avg_times_df = df_hist.groupby(['h3_origin', 'h3_destino'], as_index=False)['tiempoestimada'].median()
    avg_times_df.rename(columns={'tiempoestimada': 'avg_travel_time_min'}, inplace=True)

    # *** Optimización: Crear diccionario para búsqueda rápida ***
    h3_time_lookup = {}
    for _, row in avg_times_df.iterrows():
        # Usar tupla como clave para el diccionario
        h3_time_lookup[(row['h3_origin'], row['h3_destino'])] = int(round(row['avg_travel_time_min']))
    print(f"Diccionario de búsqueda rápida creado con {len(h3_time_lookup)} entradas H3 origen-destino.")

    print("Calculando velocidad promedio global (fallback)...")
    # Calcular velocidad solo para filas con distancia y tiempo válidos
    df_hist['speed_kmh'] = (df_hist['distance_km'] / df_hist['tiempoestimada']) * 60
    # Filtrar velocidades poco realistas antes de calcular la mediana
    df_hist_valid_speed = df_hist[(df_hist['speed_kmh'] > 1) & (df_hist['speed_kmh'] < 120)]

    if not df_hist_valid_speed.empty:
        avg_speed_kmh = df_hist_valid_speed['speed_kmh'].median()
        print(f"Velocidad promedio (mediana) calculada (fallback): {avg_speed_kmh:.2f} km/h")
    else:
        avg_speed_kmh = DEFAULT_AVG_SPEED_KMH
        print(f"No se pudieron calcular velocidades válidas. Usando fallback por defecto: {avg_speed_kmh} km/h")

    end_time = time.time()
    print(f"Pre-cómputo finalizado en {end_time - start_time:.2f} segundos.")


def get_travel_time_minutes(h3_origin, h3_dest, fallback_distance_km):
    """
    Obtiene el tiempo de viaje en minutos entre H3 usando el diccionario precalculado.
    Usa fallback basado en distancia y velocidad promedio si no se encuentra en el diccionario.
    """
    global h3_time_lookup, avg_speed_kmh

    # Intenta buscar en el diccionario (búsqueda rápida O(1) promedio)
    travel_time = h3_time_lookup.get((h3_origin, h3_dest))

    if travel_time is not None:
        return travel_time # Devuelve el tiempo precalculado si existe

    # Fallback: Calcular tiempo basado en distancia y velocidad promedio
    if pd.notna(fallback_distance_km) and fallback_distance_km > 0 and avg_speed_kmh > 0:
        calculated_time = (fallback_distance_km / avg_speed_kmh) * 60
        # Asegurarse de devolver un entero y evitar tiempos cero si la distancia es muy pequeña
        return max(1, int(round(calculated_time)))

    # Si no hay datos suficientes para el fallback, devolver un tiempo por defecto (ej. 60 min)
    # print(f"WARN: Fallback para {h3_origin}-{h3_dest} sin distancia ({fallback_distance_km}) o velocidad ({avg_speed_kmh}). Usando 60min.")
    return 60

# ----------------------------------------------------------
# Parte 4: Funciones de OR-Tools y Ejecución Principal (Optimizada)
# ----------------------------------------------------------

def process_and_print_solution(num_jobs, tasks, data, manager, routing, solution):
    """
    Procesa la solución, la imprime de forma resumida y devuelve datos estructurados.
    (Misma funcionalidad que antes, sólo se ajusta para la estructura de 'tasks')
    """
    global ID_COLUMN_NAME

    print(f"\nObjective (Total Time Minimizado): {solution.ObjectiveValue()} min")
    time_dimension = routing.GetDimensionOrDie("Time")
    jobs_dimension = routing.GetDimensionOrDie("Jobs") # Puede ser None si falló la creación

    solution_routes = [] # Lista para almacenar la información de cada ruta usada
    assigned_job_node_indices = set() # Para rastrear qué trabajos se asignaron (índices 0 a num_jobs-1)
    total_jobs_assigned_count = 0
    total_route_time_sum = 0 # Suma del tiempo de las rutas usadas

    print("\n--- Rutas Asignadas ---")
    vehicles_used = 0
    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue

        vehicles_used += 1
        route_nodes_indices = [] # Indices de los trabajos en esta ruta
        route_details_list = [] # Lista de diccionarios para los detalles de esta ruta
        index = routing.Start(vehicle_id)
        route_output = f"Ruta Movil #{vehicles_used} (OR-Tools Veh {vehicle_id}): "
        current_job_count = 0
        route_time = 0

        while not routing.IsEnd(index):
            node_index_internal = index
            node_index_mapped = manager.IndexToNode(node_index_internal) # Mapea al índice 0..num_jobs-1 (+ source/sink)

            # Solo procesar nodos de trabajo (ignorar source)
            if 0 <= node_index_mapped < num_jobs:
                job_idx = node_index_mapped # Índice del trabajo (0 a num_jobs-1)
                task_info = tasks[job_idx] # Acceder a la tarea correcta por su índice
                time_var = time_dimension.CumulVar(node_index_internal)
                arrival_time_min = solution.Min(time_var)

                job_id_display = task_info.get(ID_COLUMN_NAME, f"JobIndex_{job_idx}")
                route_output += f"{job_id_display} (Arr:{arrival_time_min}m) -> "
                current_job_count += 1

                # Guardar detalles del trabajo asignado
                job_detail = {
                    "movil_id_asignado": vehicles_used, # ID Secuencial 1..N
                    "or_tools_vehicle_id": vehicle_id, # ID interno de OR-Tools
                    "job_id_unique": job_id_display, # El ID original del trabajo
                    "node_index": job_idx, # Índice 0..num_jobs-1
                    "pickup_time_original": task_info['pickup_time_str'],
                    "arrival_minutes_solution": arrival_time_min,
                    # Añadir otros datos relevantes de task_info
                    "estimated_payment": task_info.get('estimated_payment', 0),
                    "pickup_lat": task_info.get('pickup_lat'), "pickup_lon": task_info.get('pickup_lon'),
                    "dropoff_lat": task_info.get('dropoff_lat'), "dropoff_lon": task_info.get('dropoff_lon'),
                    "job_duration_estimated_min": task_info.get('job_duration_min')
                }
                route_details_list.append(job_detail)
                route_nodes_indices.append(job_idx)
                assigned_job_node_indices.add(job_idx) # Marcar como asignado (usa índice 0..N-1)

            # Moverse al siguiente nodo en la ruta del vehículo
            index = solution.Value(routing.NextVar(index))

            # Si el siguiente es el final, obtener tiempo total de la ruta
            if routing.IsEnd(index):
                 time_var_end = time_dimension.CumulVar(index)
                 route_time = solution.Min(time_var_end)
                 total_route_time_sum += route_time # Acumular para el resumen

        route_output += f"End (Total: {route_time} min, Trabajos: {current_job_count})"
        print(route_output)

        # Guardar información de la ruta completa
        solution_routes.append({
            "movil_id_asignado": vehicles_used,
            "or_tools_vehicle_id": vehicle_id,
            "route_nodes_indices": route_nodes_indices, # Lista de índices de trabajos
            "route_details": route_details_list, # Lista de detalles por trabajo
            "total_route_time_min": route_time,
            "total_jobs_in_route": current_job_count
        })
        total_jobs_assigned_count += current_job_count # Contar trabajos

    # Devolver los datos procesados
    # total_jobs_assigned_count = len(assigned_job_node_indices) # Alternativa si solo contamos unicos
    return solution_routes, assigned_job_node_indices, total_jobs_assigned_count, vehicles_used, total_route_time_sum


def main_ortools(df_hist_path, df_pred_path):
    """
    Función principal optimizada: carga datos, configura, resuelve y exporta.
    """
    global ID_COLUMN_NAME, h3_time_lookup, avg_speed_kmh # Acceder a globales

    main_start_time = time.time()

    # 1. Cargar Datos
    print("Cargando archivos CSV...")
    try:
        df_hist = pd.read_csv(df_hist_path)
        # Especificar dtypes puede acelerar la carga y evitar errores
        dtypes_pred = {'latrecogida': float, 'lonrecogida': float,
                       'latdestino': float, 'londestino': float,
                       ID_COLUMN_NAME: str} # Asegurar que ID sea string si es necesario
        df_pred = pd.read_csv(df_pred_path, dtype=dtypes_pred, low_memory=False)
        print(f"Históricos cargados: {len(df_hist)} filas")
        print(f"Predicciones cargadas: {len(df_pred)} filas")
    except FileNotFoundError as e:
        print(f"Error CRÍTICO: No se encontró el archivo {e.filename}")
        return
    except KeyError as e:
        print(f"Error CRÍTICO: Falta la columna '{e}' requerida en el dtype. Verifica el nombre '{ID_COLUMN_NAME}'.")
        return
    except Exception as e:
        print(f"Error CRÍTICO cargando archivos CSV: {e}")
        return

    # 2. Precomputar Promedios Históricos (Usa función optimizada)
    try:
        precompute_historical_averages(df_hist)
    except ValueError as e:
        print(f"Error CRÍTICO en precomputo histórico: {e}")
        return
    except Exception as e:
        print(f"Error CRÍTICO inesperado en precomputo histórico: {e}")
        return

    # 3. Validar y Preparar Datos de Predicción (Optimizado)
    print("Validando y preparando datos de predicción...")
    prep_start_time = time.time()
    required_pred_cols = {'latrecogida', 'lonrecogida', 'latdestino', 'londestino', 'pickup_datetime', ID_COLUMN_NAME}
    if not required_pred_cols.issubset(df_pred.columns):
         missing_cols = required_pred_cols - set(df_pred.columns)
         print(f"Error CRÍTICO: Faltan columnas requeridas en predicciones: {missing_cols}")
         print(f"Columnas encontradas: {list(df_pred.columns)}")
         return

    # Asegurar columna de pago estimado
    if 'estimated_payment' not in df_pred.columns:
        print("Advertencia: Columna 'estimated_payment' no encontrada. Se usará 0.")
        df_pred['estimated_payment'] = 0
    else:
        df_pred['estimated_payment'] = pd.to_numeric(df_pred['estimated_payment'], errors='coerce').fillna(0)

    # Convertir fecha/hora y manejar errores
    try:
        df_pred['HoraFecha'] = pd.to_datetime(df_pred['pickup_datetime'], errors='coerce')
    except Exception as e:
        print(f"Error CRÍTICO convirtiendo 'pickup_datetime' a fecha: {e}")
        return

    # Filtrar filas con datos inválidos esenciales *antes* de cálculos costosos
    initial_pred_rows = len(df_pred)
    cols_to_check_na = ['HoraFecha', 'latrecogida', 'lonrecogida', 'latdestino', 'londestino', ID_COLUMN_NAME]
    df_pred_valid = df_pred.dropna(subset=cols_to_check_na).copy() # Usar .copy() para evitar SettingWithCopyWarning
    if df_pred_valid.empty:
        print(f"Error CRÍTICO: No hay trabajos válidos después de eliminar NaNs en columnas clave. {initial_pred_rows} filas iniciales.")
        return
    skipped_initial = initial_pred_rows - len(df_pred_valid)
    print(f"  {skipped_initial} trabajos iniciales descartados por NaNs en columnas clave.")

    # Calcular Horizonte de Tiempo
    start_horizon_time = df_pred_valid['HoraFecha'].min()
    print(f"Horizonte de planificación inicia en: {start_horizon_time}")

    # Calcular columnas necesarias de forma más eficiente
    print("Calculando H3, distancias, duraciones y ventanas de tiempo...")
    # Calcular H3 (usando apply)
    df_pred_valid['H3_pickup'] = df_pred_valid.apply(lambda row: simulate_h3_str(row['latrecogida'], row['lonrecogida']), axis=1)
    df_pred_valid['H3_dropoff'] = df_pred_valid.apply(lambda row: simulate_h3_str(row['latdestino'], row['londestino']), axis=1)

    # Calcular distancia fallback (apply sigue siendo razonable aquí)
    df_pred_valid['distance_job_km'] = df_pred_valid.apply(lambda row: haversine(row['latrecogida'], row['lonrecogida'], row['latdestino'], row['londestino']), axis=1)

    # Calcular duración del trabajo (usa función get_travel_time optimizada)
    df_pred_valid['job_duration_min'] = df_pred_valid.apply(
        lambda row: get_travel_time_minutes(row['H3_pickup'], row['H3_dropoff'], row['distance_job_km']), axis=1
    )

    # Calcular ventanas de tiempo (vectorizado)
    time_diff_seconds = (df_pred_valid['HoraFecha'] - start_horizon_time).dt.total_seconds()
    df_pred_valid['pickup_start_min'] = (time_diff_seconds / 60).round().astype(int)
    # Asegurar que no sea negativo si alguna fecha es anterior al mínimo (raro pero posible)
    df_pred_valid['pickup_start_min'] = df_pred_valid['pickup_start_min'].clip(lower=0)
    df_pred_valid['pickup_end_min'] = df_pred_valid['pickup_start_min'] + PICKUP_WINDOW_DURATION_MINUTES
    df_pred_valid['pickup_time_str'] = df_pred_valid['HoraFecha'].dt.strftime('%Y-%m-%d %H:%M')

    # Filtrar trabajos cuya duración calculada sea inválida (ej. si fallback falló)
    final_valid_rows = len(df_pred_valid)
    df_pred_valid = df_pred_valid.dropna(subset=['H3_pickup', 'H3_dropoff', 'distance_job_km', 'job_duration_min'])
    df_pred_valid = df_pred_valid[df_pred_valid['job_duration_min'] < INF_TIME] # Filtrar duraciones infinitas
    skipped_calculation = final_valid_rows - len(df_pred_valid)
    if skipped_calculation > 0:
        print(f"  Advertencia: Se descartaron {skipped_calculation} trabajos adicionales debido a problemas en cálculo de H3/distancia/duración.")

    num_jobs = len(df_pred_valid)
    if num_jobs == 0:
        print(f"Error CRÍTICO: No quedaron trabajos válidos después de todos los cálculos. Total inicial: {initial_pred_rows}, descartados: {skipped_initial + skipped_calculation}")
        return
    print(f"Número final de trabajos a asignar: {num_jobs}")

    # *** Optimización: Crear lista de tareas desde DataFrame ***
    tasks = df_pred_valid.to_dict('records') # Mucho más rápido que iterrows

    # Extraer datos necesarios para la matriz de tiempos en listas para acceso rápido
    job_durations_min = df_pred_valid['job_duration_min'].tolist()
    pickup_coords = list(zip(df_pred_valid['latrecogida'], df_pred_valid['lonrecogida']))
    dropoff_coords = list(zip(df_pred_valid['latdestino'], df_pred_valid['londestino']))
    h3_pickups = df_pred_valid['H3_pickup'].tolist()
    h3_dropoffs = df_pred_valid['H3_dropoff'].tolist()
    pickup_windows_min = list(zip(df_pred_valid['pickup_start_min'], df_pred_valid['pickup_end_min']))

    prep_end_time = time.time()
    print(f"Preparación de datos finalizada en {prep_end_time - prep_start_time:.2f} segundos.")


    # 4. Configurar Nodos y Vehículos
    num_vehicles = min(PARAM_MAX_MOVILES, num_jobs) # No usar más vehículos que trabajos
    print(f"Número de vehículos a usar: {num_vehicles}")
    num_locations_total = num_jobs + 2 # Trabajos + Source + Sink
    source_node = num_jobs # Índice del nodo de origen
    sink_node = num_jobs + 1 # Índice del nodo de destino (sumidero)

    # 5. Precalcular Matriz de Tiempos (Optimizada internamente)
    print("Precalculando matriz de tiempos de viaje inter-trabajos...")
    matrix_start_time = time.time()
    # Inicializar con un valor grande (o 0 si se maneja en callback)
    travel_times_min_matrix = [[0] * num_jobs for _ in range(num_jobs)]

    # El bucle N^2 es necesario, pero el interior es más rápido ahora
    for i in tqdm(range(num_jobs), desc="Calculando Matriz Tiempos", unit="job", ncols=100):
        for j in range(num_jobs):
            if i == j:
                # No se puede viajar de un trabajo a sí mismo instantáneamente
                travel_times_min_matrix[i][j] = INF_TIME
                continue

            # Obtener coordenadas y H3 precalculados de las listas
            lat1, lon1 = dropoff_coords[i]
            lat2, lon2 = pickup_coords[j]
            h3_origin = h3_dropoffs[i] # El origen del viaje es el dropoff del trabajo anterior
            h3_dest = h3_pickups[j]    # El destino del viaje es el pickup del siguiente

            # Calcular distancia solo como fallback para get_travel_time_minutes
            distance_km_fallback = haversine(lat1, lon1, lat2, lon2)

            # Usar la función optimizada que busca en el diccionario primero
            travel_times_min_matrix[i][j] = get_travel_time_minutes(h3_origin, h3_dest, distance_km_fallback)

    matrix_end_time = time.time()
    print(f"Matriz de tiempos inter-trabajos calculada en {matrix_end_time - matrix_start_time:.2f} segundos.")

    # 6. Crear Manager y Modelo OR-Tools
    print("Configurando modelo OR-Tools..."); model_start_time = time.time()
    try:
        # Nodos de inicio (todos desde source) y fin (todos hacia sink)
        start_nodes = [source_node] * num_vehicles
        end_nodes = [sink_node] * num_vehicles
        manager = pywrapcp.RoutingIndexManager(num_locations_total, num_vehicles, start_nodes, end_nodes)
        routing = pywrapcp.RoutingModel(manager); print("Manager y Modelo creados.")
    except Exception as e:
        print(f"Error CRÍTICO creando modelo OR-Tools: {e}"); return

    # 7. Callback de Tránsito (Tiempo)
    # Esta función ahora combina el tiempo de viaje inter-trabajo y la duración del trabajo origen
    def time_callback(from_index, to_index):
        """Devuelve el tiempo total (viaje + servicio en origen) para ir de from_index a to_index."""
        try:
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)

            # Determinar tiempo de viaje entre nodos
            travel_time = 0
            if from_node == source_node and 0 <= to_node < num_jobs:
                # Viaje desde el origen a la primera recogida: asumir tiempo 0 o un tiempo fijo si se conoce
                # Podríamos calcular tiempo desde un depósito central a pickup_coords[to_node] si tuviéramos la ubicación del depósito
                travel_time = 0 # Simplificación: tiempo de viaje desde source es 0
            elif 0 <= from_node < num_jobs and 0 <= to_node < num_jobs:
                # Viaje entre dos trabajos (usa la matriz precalculada)
                travel_time = travel_times_min_matrix[from_node][to_node]
            elif 0 <= from_node < num_jobs and to_node == sink_node:
                # Viaje desde la última entrega al destino final (sink): asumir tiempo 0 o fijo
                 travel_time = 0 # Simplificación: tiempo de viaje hacia sink es 0
            elif from_node == source_node and to_node == sink_node:
                 # Caso de vehículo no usado (source a sink directo)
                 travel_time = 0
            # else: # Otros casos (sink a source, sink a job, job a source) no deberían ocurrir en rutas válidas

            # Obtener duración del servicio en el nodo de origen (si es un trabajo)
            service_time = 0
            if 0 <= from_node < num_jobs:
                service_time = job_durations_min[from_node]

            # Tiempo total = tiempo de viaje + tiempo de servicio en el nodo origen
            total_time = travel_time + service_time

            # Asegurarse de que no exceda INF_TIME y sea entero
            return min(int(round(total_time)), INF_TIME)

        except IndexError:
             # print(f"WARN: IndexError en time_callback({from_index}, {to_index}) -> Nodes ({from_node}, {to_node})")
             return INF_TIME # Penalización alta si algo falla
        except Exception as e:
             # print(f"WARN: Exception en time_callback: {e}")
             return INF_TIME

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    # Establecer el costo de los arcos = tiempo (queremos minimizar tiempo total)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    print("Callback de tiempo registrado y costo de arcos establecido.")

    # 8. Dimensión de Tiempo (Restricciones de Ventana y Duración de Ruta) - CORREGIDO
    print("Añadiendo dimensión de Tiempo..."); time_dimension_name = "Time"

    # Calcular el tiempo máximo de ruta permitido por vehículo
    max_route_time_min = PARAM_MAX_HORAS_POR_MOVIL * 60

    # Calcular un horizonte global suficientemente grande
    # Debe ser al menos el fin de la ventana más tardía + duración + slack, o simplemente un valor grande seguro.
    latest_window_end = max(w[1] for w in pickup_windows_min) if pickup_windows_min else 0
    longest_job = max(job_durations_min) if job_durations_min else 0
    # Un horizonte seguro sería el último momento posible para terminar un trabajo + un margen
    horizon = latest_window_end + longest_job + MAX_SLACK_MINUTES
    # O simplemente un valor muy grande si confiamos en max_route_time_min para limitar
    # horizon = INF_TIME # Otra opción si las ventanas son razonables

    print(f"  Capacidad de Tiempo por Ruta (Vehículo): {max_route_time_min} min")
    print(f"  Horizonte Global Máximo del Plan: {horizon} min")
    print(f"  Slack máximo permitido por parada: {MAX_SLACK_MINUTES} min")

    # --- LLAMADA CORREGIDA A AddDimension ---
    routing.AddDimension(
        transit_callback_index,
        MAX_SLACK_MINUTES,     # Slack máximo (tiempo de espera en cada nodo)
        max_route_time_min,    # Capacidad: Tiempo MÁXIMO ACUMULADO por RUTA de vehículo
        False,                 # NO empezar tiempo acumulado en cero para nodos de inicio
        time_dimension_name
    )
    # -----------------------------------------
    time_dimension = routing.GetDimensionOrDie(time_dimension_name); print(f"Dimensión '{time_dimension_name}' añadida.")

    print("Aplicando ventanas de tiempo a los trabajos...");
    # Aplicar ventanas de recogida a cada nodo de trabajo
    try:
        for job_idx in range(num_jobs):
            index = manager.NodeToIndex(job_idx)
            start_win, end_win = pickup_windows_min[job_idx]

            # Comprobación adicional (opcional pero útil para debug)
            # if start_win > max_route_time_min:
            #     print(f"WARN: Job {job_idx} start window {start_win} potentially > max route time {max_route_time_min}")
            # if end_win > horizon:
            #     print(f"WARN: Job {job_idx} end window {end_win} > global horizon {horizon}")

            if start_win > end_win:
                 job_id_display = tasks[job_idx].get(ID_COLUMN_NAME, job_idx)
                 print(f"Advertencia: Ventana inválida para Job {job_id_display}: [{start_win}, {end_win}]. Ajustando a [{start_win}, {start_win}].")
                 end_win = start_win

            # Intentar aplicar la ventana de tiempo
            # print(f"DEBUG: Setting range for Job {job_idx} (Node {index}): [{start_win}, {end_win}]") # Descomentar para debug
            time_dimension.CumulVar(index).SetRange(start_win, end_win)

        # Aplicar ventanas a los nodos de inicio/fin de los vehículos
        # Permite que las rutas empiecen/terminen en cualquier momento dentro del horizonte global
        for vehicle_id in range(num_vehicles):
             start_index = routing.Start(vehicle_id)
             end_index = routing.End(vehicle_id)
             # print(f"DEBUG: Setting range for Start Node Veh {vehicle_id}: [0, {horizon}]") # Descomentar para debug
             time_dimension.CumulVar(start_index).SetRange(0, horizon)
             # print(f"DEBUG: Setting range for End Node Veh {vehicle_id}: [0, {horizon}]") # Descomentar para debug
             time_dimension.CumulVar(end_index).SetRange(0, horizon) # El tiempo de finalización también debe estar dentro del horizonte

        print("Ventanas de tiempo aplicadas.")
    except Exception as e:
        # Captura más específica si el error persiste aquí
        job_id_display = tasks[job_idx].get(ID_COLUMN_NAME, job_idx) # job_idx podría ser el último del bucle
        print(f"\n!!! ERROR CRÍTICO al aplicar ventana de tiempo [{start_win}, {end_win}] para Job con índice {job_idx} ({job_id_display}): {e} !!!")
        print(f"    Revisa si la ventana es compatible con max_route_time_min={max_route_time_min} y horizon={horizon}")
        return # Detener la ejecución si falla aquí

    # 9. Dimensión de Conteo de Trabajos (Opcional, pero útil)
    print(f"Añadiendo dimensión de Conteo de Trabajos (max: {PARAM_MAX_RESERVAS_POR_MOVIL})...")
    jobs_dimension_name = "Jobs";
    try:
        # Callback: devuelve 1 para cada nodo de trabajo, 0 para source/sink
        def demand_callback(from_index):
            from_node = manager.IndexToNode(from_index)
            return 1 if 0 <= from_node < num_jobs else 0

        demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
        # Capacidad de cada vehículo = máximo número de trabajos permitidos
        vehicle_capacities = [PARAM_MAX_RESERVAS_POR_MOVIL] * num_vehicles
        routing.AddDimensionWithVehicleCapacity(
            demand_callback_index,
            0,  # Slack (no aplica a conteo)
            vehicle_capacities,
            True,  # Empezar acumulado en cero
            jobs_dimension_name
        )
        print(f"Dimensión '{jobs_dimension_name}' añadida.")
    except Exception as e:
        print(f"Advertencia: Error añadiendo dimensión '{jobs_dimension_name}': {e}. Continuará sin ella.")


    # 10. Configurar Búsqueda y Resolver
    print("Configurando parámetros de búsqueda y resolviendo...")
    model_end_time = time.time()
    print(f"Configuración del modelo OR-Tools completada en {model_end_time - model_start_time:.2f} segundos.")

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # Estrategia para encontrar la primera solución rápidamente
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC # Buenas opciones: PATH_CHEAPEST_ARC, PARALLEL_CHEAPEST_INSERTION
    )
    # Metaheurística para mejorar la solución inicial
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH # Buenas opciones: GUIDED_LOCAL_SEARCH, TABU_SEARCH
    )
    # Límite de tiempo para la búsqueda
    search_parameters.time_limit.seconds = SOLVER_TIME_LIMIT_SECONDS
    # Loguear progreso de la búsqueda (útil para ver si se atasca)
    search_parameters.log_search = True # Poner en False para menos output

    print(f"Límite de tiempo del solver: {SOLVER_TIME_LIMIT_SECONDS} segundos. Iniciando Solve()...")
    solve_start_time = time.time()
    solution = routing.SolveWithParameters(search_parameters)
    solve_end_time = time.time()
    print(f"Proceso de Solve() finalizado en {solve_end_time - solve_start_time:.2f} segundos.")

    # 11. Procesar Solución, Exportar y Mostrar Resumen
    if solution:
        print("\n********************* Solución Encontrada! *********************")
        # --- Procesar la solución para obtener datos ---
        solution_routes, assigned_job_node_indices, total_jobs_assigned, vehicles_used, total_route_time_sum = process_and_print_solution(
            num_jobs, tasks, {"num_vehicles": num_vehicles, "source_node": source_node, "sink_node": sink_node},
            manager, routing, solution
        )

        # --- Crear DataFrame de rutas asignadas ---
        rutas_asignadas_list = []
        for route in solution_routes:
            # Añadir cada detalle de trabajo de la ruta a la lista general
            rutas_asignadas_list.extend(route['route_details'])

        df_rutas = pd.DataFrame(rutas_asignadas_list)
        if not df_rutas.empty:
             # Opcional: Ordenar por móvil y luego por hora de llegada
             df_rutas.sort_values(by=['movil_id_asignado', 'arrival_minutes_solution'], inplace=True)
        print(f"\nDataFrame 'df_rutas' creado con {len(df_rutas)} filas (trabajos asignados).")

        # --- Crear DataFrame de trabajos NO asignados ---
        reservas_no_asignadas_list = []
        all_original_indices = set(range(num_jobs)) # Indices 0 a num_jobs-1
        unassigned_indices = all_original_indices - assigned_job_node_indices

        for job_idx in unassigned_indices:
            unassigned_task = tasks[job_idx].copy() # Obtener datos del trabajo original
            # Añadir motivo (simplificado)
            unassigned_task['motivo_no_asignado'] = "No incluido en solución OR-Tools (restricciones/tiempo)"
            # Limpiar datos internos si no se quieren en el CSV
            # keys_to_remove = ['pickup_start_min', 'pickup_end_min', ...]
            reservas_no_asignadas_list.append(unassigned_task)

        df_no_asignadas = pd.DataFrame(reservas_no_asignadas_list)
        print(f"DataFrame 'df_no_asignadas' creado con {len(df_no_asignadas)} filas (trabajos no asignados).")

        # --- Exportar a CSV ---
        timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        rutas_csv_path = f"rutas_asignadas_ortools_{timestamp_str}.csv"
        no_asignadas_csv_path = f"reservas_no_asignadas_ortools_{timestamp_str}.csv"
        try:
            if not df_rutas.empty:
                 df_rutas.to_csv(rutas_csv_path, index=False, encoding='utf-8-sig') # utf-8-sig para mejor compatibilidad Excel
                 print(f"Resultados de rutas asignadas exportados a: {rutas_csv_path}")
            else:
                 print("No se generó archivo de rutas asignadas (ningún trabajo asignado).")

            if not df_no_asignadas.empty:
                 df_no_asignadas.to_csv(no_asignadas_csv_path, index=False, encoding='utf-8-sig')
                 print(f"Resultados de reservas no asignadas exportados a: {no_asignadas_csv_path}")
            else:
                 print("No se generó archivo de reservas no asignadas (todos los trabajos asignados).")


            # --- Descargar en Google Colab (si aplica) ---
            if IN_COLAB:
                print("Iniciando descarga de archivos en Colab...")
                if not df_rutas.empty: files.download(rutas_csv_path)
                if not df_no_asignadas.empty: files.download(no_asignadas_csv_path)
                print("Descarga en Colab solicitada.")
            # --------------------------------------------

        except Exception as e:
            print(f"\nError al exportar archivos CSV: {e}")

        # --- Imprimir Resumen Final ---
        print("\n--- Resumen Final de la Optimización ---")
        total_payment_assigned = df_rutas['estimated_payment'].sum() if not df_rutas.empty and 'estimated_payment' in df_rutas.columns else 0
        avg_jobs_per_movil = total_jobs_assigned / vehicles_used if vehicles_used > 0 else 0
        avg_time_per_movil = total_route_time_sum / vehicles_used if vehicles_used > 0 else 0

        print(f"Total Trabajos Input Válidos: {num_jobs}")
        print(f"Móviles Utilizados:           {vehicles_used} / {num_vehicles}")
        print(f"Trabajos Asignados:           {total_jobs_assigned}")
        print(f"Trabajos NO Asignados:        {len(df_no_asignadas)}")
        print(f"Objetivo OR-Tools (Tiempo):   {solution.ObjectiveValue()} min (Suma tiempos acumulados en nodos finales)")
        # print(f"Suma Tiempos Rutas Usadas:    {total_route_time_sum} min") # Puede diferir del objetivo dependiendo de la definición exacta
        print(f"Promedio Trabajos/Móvil:      {avg_jobs_per_movil:.2f}")
        print(f"Promedio Tiempo/Móvil:        {avg_time_per_movil:.2f} min")
        # Formatear el pago total como moneda local (ejemplo CLP)
        try:
             print(f"Ganancia Total Estimada:      ${total_payment_assigned:,.0f} CLP")
        except ValueError:
             print(f"Ganancia Total Estimada:      {total_payment_assigned}") # Sin formato si falla

    else:
        # --- Si no se encontró solución ---
        print("\n******************** No se encontró solución ********************")
        print("El solver de OR-Tools no pudo encontrar una asignación factible dentro del tiempo límite y con las restricciones dadas.")
        print("Posibles causas:")
        print("  - Restricciones muy estrictas (ventanas de tiempo, capacidad móvil, tiempo máximo ruta).")
        print("  - Datos inconsistentes o geográficamente dispersos.")
        print("  - Límite de tiempo del solver (`SOLVER_TIME_LIMIT_SECONDS`) insuficiente.")
        print("  - Número insuficiente de vehículos (`PARAM_MAX_MOVILES`).")
        print("  - Errores en la definición del modelo o callbacks.")
        print("No se generaron archivos de resultados.")

    main_end_time = time.time()
    print(f"\nProceso completo de optimización finalizado en {main_end_time - main_start_time:.2f} segundos.")
    print("**********************************************************\n")


# ----------------------------------------------------------
# Parte 5: Bloque de Ejecución Principal
# ----------------------------------------------------------
if __name__ == "__main__":
      # --- ¡¡¡ CONFIGURA LAS RUTAS A TUS ARCHIVOS CSV !!! ---
      hist_file = '/content/distancias H3 2.0 (Hist).csv'        # Ruta al archivo histórico
      pred_file = '/content/distancias H3 2.0 (Pred) - 25-04 05-15.csv'    # Ruta al archivo de predicciones/trabajos
      # ---------------------------------------------------------

      print(f"Iniciando proceso de optimización de rutas...")
      # --- LÍNEA CORREGIDA ---
      try:
            print(f"Usando OR-Tools versión: {ortools.__version__}")
      except AttributeError:
            print("No se pudo determinar la versión de OR-Tools automáticamente.") # Fallback por si acaso
      # -----------------------
      print(f"Archivo histórico:        {hist_file}")
      print(f"Archivo de predicciones:  {pred_file}")
      print(f"Columna ID de trabajo:    '{ID_COLUMN_NAME}'")
      print("-" * 50)

      # Verificar existencia de archivos antes de empezar
      if not os.path.exists(hist_file):
          print(f"\nERROR FATAL: El archivo histórico '{hist_file}' no existe.")
          print("Por favor, verifica la ruta y el nombre del archivo.")
      elif not os.path.exists(pred_file):
          print(f"\nERROR FATAL: El archivo de predicciones '{pred_file}' no existe.")
          print("Por favor, verifica la ruta y el nombre del archivo.")
      else:
          # Ejecutar la función principal
          main_ortools(hist_file, pred_file)

      print("Script finalizado.")

Iniciando proceso de optimización de rutas...
Usando OR-Tools versión: 9.12.4544
Archivo histórico:        /content/distancias H3 2.0 (Hist).csv
Archivo de predicciones:  /content/distancias H3 2.0 (Pred) - 25-04 05-15.csv
Columna ID de trabajo:    'job_id'
--------------------------------------------------
Cargando archivos CSV...
Históricos cargados: 461962 filas
Predicciones cargadas: 215 filas
Iniciando pre-cómputo de promedios históricos...
Limpiando datos históricos...
  85796 filas eliminadas por NaNs o tiempo inválido.
Calculando H3 simulados y distancias para datos históricos...


<ipython-input-13-a4c12e4e2a30>:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hist['tiempoestimada'] = pd.to_numeric(df_hist['tiempoestimada'], errors='coerce')


  7 filas eliminadas por cálculo inválido de H3/distancia.
Calculando tiempos promedio (mediana) agrupados por H3...
Diccionario de búsqueda rápida creado con 79205 entradas H3 origen-destino.
Calculando velocidad promedio global (fallback)...
Velocidad promedio (mediana) calculada (fallback): 36.10 km/h
Pre-cómputo finalizado en 23.30 segundos.
Validando y preparando datos de predicción...
  0 trabajos iniciales descartados por NaNs en columnas clave.
Horizonte de planificación inicia en: 2025-04-25 05:00:00
Calculando H3, distancias, duraciones y ventanas de tiempo...
Número final de trabajos a asignar: 215
Preparación de datos finalizada en 0.03 segundos.
Número de vehículos a usar: 100
Precalculando matriz de tiempos de viaje inter-trabajos...


Calculando Matriz Tiempos: 100%|████████████████████████████████| 215/215 [00:00<00:00, 288.43job/s]

Matriz de tiempos inter-trabajos calculada en 0.75 segundos.
Configurando modelo OR-Tools...
Manager y Modelo creados.
Callback de tiempo registrado y costo de arcos establecido.
Añadiendo dimensión de Tiempo...
  Capacidad de Tiempo por Ruta (Vehículo): 600 min
  Horizonte Global Máximo del Plan: 967 min
  Slack máximo permitido por parada: 30 min
Dimensión 'Time' añadida.
Aplicando ventanas de tiempo a los trabajos...

!!! ERROR CRÍTICO al aplicar ventana de tiempo [620, 680] para Job con índice 208 (23673492): CP Solver fail !!!
    Revisa si la ventana es compatible con max_route_time_min=600 y horizon=967
Script finalizado.
